In [5]:
!pip install yt-dlp pydub streamlit -q
!npm install -g localtunnel -q
!apt-get install ffmpeg -y -q

⠙⠹⠸⠼⠴⠦
changed 22 packages in 809ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [6]:
%%writefile 102313005.py
import sys
import os
import yt_dlp
from pydub import AudioSegment

def mashup(singer, n, y, output_file):
    search_query = f"ytsearch{n}:{singer}"
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_video_%(id)s.%(ext)s',
        'noplaylist': True,
        'quiet': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }],
    }

    print(f"Downloading {n} videos of {singer}...")
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([search_query])

    combined = AudioSegment.empty()
    audio_files = sorted([f for f in os.listdir('.') if f.startswith('temp_video_') and f.endswith('.mp3')])

    print(f"Trimming first {y} seconds and merging...")
    for file in audio_files[:n]:
        audio = AudioSegment.from_file(file)
        trimmed = audio[:y * 1000]
        combined += trimmed
        os.remove(file)

    combined.export(output_file, format="mp3")
    print(f"Success! Mashup saved as {output_file}")

if __name__ == "__main__":
    if len(sys.argv) != 5:
        print("Usage: python <program.py> <SingerName> <NumberOfVideos> <AudioDuration> <OutputFileName>")
        sys.exit(1)

    try:
        singer_name = sys.argv[1]
        num_videos = int(sys.argv[2])
        duration = int(sys.argv[3])
        output = sys.argv[4]

        if num_videos <= 10:
            print("Error: Number of videos (N) must be greater than 10.")
        elif duration <= 20:
            print("Error: Audio duration (Y) must be greater than 20.")
        else:
            mashup(singer_name, num_videos, duration, output)

    except Exception as e:
        print(f"Error: {e}")

Overwriting 101556.py


In [11]:
!pip install gradio yt-dlp pydub -q
!apt-get install ffmpeg -y -q

Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [13]:
import gradio as gr
import os, zipfile, smtplib, yt_dlp
from pydub import AudioSegment
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

SENDER_EMAIL = "angad.28.03.2005@gmail.com"
APP_PASSWORD = "sgxs jgor dini kkfv"

def send_email(receiver_email, zip_path):
    msg = MIMEMultipart()
    msg['From'] = SENDER_EMAIL
    msg['To'] = receiver_email
    msg['Subject'] = "Mashup Result Zip"
    with open(zip_path, "rb") as f:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f"attachment; filename={os.path.basename(zip_path)}")
        msg.attach(part)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(SENDER_EMAIL, APP_PASSWORD)
        server.send_message(msg)

def run_mashup(singer, n_videos, duration, email_id):
    if "@" not in email_id or "." not in email_id:
        return "Error: Email id must be correct"
    if n_videos <= 10:
        return "Error: Number of videos (N) must be greater than 10"
    if duration <= 20:
        return "Error: Audio duration (Y) must be greater than 20"

    try:
        combined = AudioSegment.empty()
        ydl_opts = {
            'format': 'bestaudio/best',
            'noplaylist': True,
            'playlist_items': f'1-{int(n_videos)}',
            'quiet': True,
            'outtmpl': 'web_temp_%(id)s.%(ext)s'
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            results = ydl.extract_info(f"ytsearch{int(n_videos)}:{singer}", download=True)
            for entry in results['entries']:
                if entry is None: continue
                fname = ydl.prepare_filename(entry)
                audio = AudioSegment.from_file(fname)[:int(duration) * 1000]
                combined += audio
                if os.path.exists(fname): os.remove(fname)

        output_mp3 = "result.mp3"
        combined.export(output_mp3, format="mp3")
        zip_name = "mashup_result.zip"
        with zipfile.ZipFile(zip_name, 'w') as z:
            z.write(output_mp3)

        send_email(email_id, zip_name)

        if os.path.exists(output_mp3): os.remove(output_mp3)
        if os.path.exists(zip_name): os.remove(zip_name)

        return f"Success! Result file in zip format has been sent to {email_id}."
    except Exception as e:
        return f"An error occurred: {str(e)}"

interface = gr.Interface(
    fn=run_mashup,
    inputs=[
        gr.Textbox(label="Singer Name", value="Sharry Mann"),
        gr.Number(label="# of videos", value=20),
        gr.Number(label="duration of each video (sec)", value=30),
        gr.Textbox(label="Email Id")
    ],
    outputs="text",
    title="Program 2: Mashup Web Service",
    description="Enter details to create a mashup and receive it via email in zip format."
)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b9989c15b4877f529e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
